In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import calendar
from datetime import date

%run ../db.py
%run ../database.py
%run ../utils.py


In [ ]:
cgreen = '#27ae60'
cblue = '#2980b9'
cred = '#c0392b'
corange = '#d35400'
cgrey = '#7f8c8d'
cdark = '#2c3e50'
cwithe = '#ecf0f1'

figsize = (10, 6)

limite_suplementacao_loa = 0.15

In [ ]:
spec = {
    'data_base': 'Mês',
    'arrecadado': 'Arrecadado',
    'empenhado': 'Empenhado',
    'liquidado': 'Liquidado',
    'dif_arrec_emp': 'Empenhado / Arrecadado',
    'dif_arrec_liq': 'Liquidado / Arrecadado'
}
fmt = {
    'data_base': date_to_month_name,
    'arrecadado': money_formatter,
    'empenhado': money_formatter,
    'liquidado': money_formatter,
    'dif_arrec_emp': perc0_formatter,
    'dif_arrec_liq': perc0_formatter
}

In [ ]:
def plot_acumulado(df, titulo):
    fig, ax = plt.subplots(figsize=figsize)
    fig.suptitle(titulo, x=0, fontsize=16)
    plt.title('valores acumulados', x=0)
    x = df['data_base'].dt.strftime('%b')
    plt.plot(x, df['arrecadado'], color=cgreen, label='Arrecadado', marker='o')
    plt.plot(x, df['empenhado'], color=cred, label='Empenhado', marker='s')
    plt.plot(x, df['liquidado'], color=corange, label='Liquidado', marker='^')
    plt.ylim(bottom=0)
    ax.yaxis.set_major_formatter(int_formatter)
    plt.xticks(x)
    plt.legend(loc='best', ncol=3)
    plt.show()

def adiciona_data_base(df):
    last_day_of_month = []
    for month in range(df['data_base'].max().month + 1, 13):
        last_day_of_month.append(date(df['data_base'].max().year, month, calendar.monthrange(df['data_base'].max().year, month)[1]))
    for dt in last_day_of_month:
        n = pd.DataFrame([{'data_base': dt}])
        df = pd.concat([df, n])
    df = df.reset_index(drop=True)
    return df


# Receitas x Despesas

Nesta parte são copmradas as receitas e despesas da **Prefeitura**, excluídas, portanto, as receias do *RPPS*. Também são excluídas as receitas *intra-orçamentárias*.

As receitas são apresentadas em seus valores líquidos das deduções da receita.

## Receita Total x Despesa Total

In [ ]:
nome = 'Receita Total x Despesa Total'
sql = f'''
WITH d AS (
	SELECT
		data_final AS data_base,
		0.0 AS arrecadado,
		SUM(valor_empenhado) AS empenhado,
		SUM(valor_liquidado) AS liquidado
	FROM
		pad."BAL_DESP"
	WHERE
		entidade LIKE 'pm'
		AND controle BETWEEN {controle_ano}01 AND {controle_mes}
	GROUP BY
		data_base
	ORDER BY
		data_base ASC
), r AS (
	SELECT
		data_final AS data_base,
		SUM(receita_realizada) AS arrecadado,
		0.0 AS empenhado,
		0.0 AS liquidado
	FROM
		pad."BAL_REC"
	WHERE
		entidade LIKE 'pm'
		AND controle BETWEEN {controle_ano}01 AND {controle_mes}
		AND classe_receita NOT LIKE 'intra'
	GROUP BY
		data_base
	ORDER BY
		data_base ASC
), t AS (SELECT * FROM r UNION SELECT * FROM d)
SELECT data_base, SUM(arrecadado) AS arrecadado, SUM(empenhado) AS empenhado, SUM(liquidado) AS liquidado FROM t GROUP BY data_base ORDER BY data_base ASC
'''
#print(sql)
orcamento = sqldf(sql)
orcamento['dif_arrec_emp'] = orcamento['empenhado'] / orcamento['arrecadado']
orcamento['dif_arrec_liq'] = orcamento['liquidado'] / orcamento['arrecadado']
orcamento = adiciona_data_base(orcamento)
orcamento['data_base'] = pd.to_datetime(orcamento['data_base'], format='%Y-%m-%d')
#orcamento

In [ ]:
plot_acumulado(orcamento.copy(), nome)

In [ ]:
df = df_table(orcamento.copy(), spec, fmt)
df.style.set_caption(f'{nome} - valores acumulados - {dt.year}').hide(axis='index')
#df

## Receita Corrente x Despesa Corrente

In [ ]:
nome = 'Receita Corrente x Despesa Corrente'
sql = f'''
WITH d AS (
	SELECT
		data_final AS data_base,
		0.0 AS arrecadado,
		SUM(valor_empenhado) AS empenhado,
		SUM(valor_liquidado) AS liquidado
	FROM
		pad."BAL_DESP"
	WHERE
		entidade LIKE 'pm'
		AND controle BETWEEN {controle_ano}01 AND {controle_mes}
        AND elemento LIKE '3%%'
	GROUP BY
		data_base
	ORDER BY
		data_base ASC
), r AS (
	SELECT
		data_final AS data_base,
		SUM(receita_realizada) AS arrecadado,
		0.0 AS empenhado,
		0.0 AS liquidado
	FROM
		pad."BAL_REC"
	WHERE
		entidade LIKE 'pm'
		AND controle BETWEEN {controle_ano}01 AND {controle_mes}
		AND classe_receita NOT LIKE 'intra'
        AND receita_base LIKE '1%%'
	GROUP BY
		data_base
	ORDER BY
		data_base ASC
), t AS (SELECT * FROM r UNION SELECT * FROM d)
SELECT data_base, SUM(arrecadado) AS arrecadado, SUM(empenhado) AS empenhado, SUM(liquidado) AS liquidado FROM t GROUP BY data_base ORDER BY data_base ASC
'''
#print(sql)
orcamento = sqldf(sql)
orcamento['dif_arrec_emp'] = orcamento['empenhado'] / orcamento['arrecadado']
orcamento['dif_arrec_liq'] = orcamento['liquidado'] / orcamento['arrecadado']
orcamento = adiciona_data_base(orcamento)
orcamento['data_base'] = pd.to_datetime(orcamento['data_base'], format='%Y-%m-%d')
#orcamento

In [ ]:
plot_acumulado(orcamento.copy(), nome)

In [ ]:
df = df_table(orcamento.copy(), spec, fmt)
df.style.set_caption(f'{nome} - valores acumulados - {dt.year}').hide(axis='index')
#df

## Receita Corrente x Despesa com Pessoal e Encargos Sociais

In [ ]:
nome = 'Receita Corrente x Despesa com Pessoal e Encargos Sociais'
sql = f'''
WITH d AS (
	SELECT
		data_final AS data_base,
		0.0 AS arrecadado,
		SUM(valor_empenhado) AS empenhado,
		SUM(valor_liquidado) AS liquidado
	FROM
		pad."BAL_DESP"
	WHERE
		entidade LIKE 'pm'
		AND controle BETWEEN {controle_ano}01 AND {controle_mes}
        AND elemento LIKE '31%%'
	GROUP BY
		data_base
	ORDER BY
		data_base ASC
), r AS (
	SELECT
		data_final AS data_base,
		SUM(receita_realizada) AS arrecadado,
		0.0 AS empenhado,
		0.0 AS liquidado
	FROM
		pad."BAL_REC"
	WHERE
		entidade LIKE 'pm'
		AND controle BETWEEN {controle_ano}01 AND {controle_mes}
		AND classe_receita NOT LIKE 'intra'
        AND receita_base LIKE '1%%'
	GROUP BY
		data_base
	ORDER BY
		data_base ASC
), t AS (SELECT * FROM r UNION SELECT * FROM d)
SELECT data_base, SUM(arrecadado) AS arrecadado, SUM(empenhado) AS empenhado, SUM(liquidado) AS liquidado FROM t GROUP BY data_base ORDER BY data_base ASC
'''
#print(sql)
orcamento = sqldf(sql)
orcamento['dif_arrec_emp'] = orcamento['empenhado'] / orcamento['arrecadado']
orcamento['dif_arrec_liq'] = orcamento['liquidado'] / orcamento['arrecadado']
orcamento = adiciona_data_base(orcamento)
orcamento['data_base'] = pd.to_datetime(orcamento['data_base'], format='%Y-%m-%d')
#orcamento

In [ ]:
plot_acumulado(orcamento.copy(), nome)

In [ ]:
df = df_table(orcamento.copy(), spec, fmt)
df.style.set_caption(f'{nome} - valores acumulados - {dt.year}').hide(axis='index')
#df

## Suplementação Autorizada na LOA

In [ ]:
nome = 'Suplementação Autorizada na LOA'
sql = f'''
WITH disponivel AS (
	SELECT
		data_final AS data_base,
		SUM(saldo_final) AS vldisponivel,
		0.0 AS vlutilizado
	FROM
		pad."BAL_VER"
	WHERE
		entidade LIKE 'pm'
		AND controle BETWEEN {controle_ano}01 AND {controle_ano}12
		AND conta_contabil LIKE '899940101%%'
	GROUP BY
		data_base
	ORDER BY
		data_base ASC
), utilizado AS (
	SELECT
		data_final AS data_base,
		0.0 AS vldisponivel,
		SUM(saldo_final) AS vlutilizado
	FROM
		pad."BAL_VER"
	WHERE
		entidade LIKE 'pm'
		AND controle BETWEEN {controle_ano}01 AND {controle_ano}12
		AND conta_contabil LIKE '899940102%%'
	GROUP BY
		data_base
	ORDER BY
		data_base ASC
), total AS (
	SELECT * FROM disponivel UNION ALL SELECT * FROM utilizado
)
SELECT
	data_base,
	SUM(vldisponivel) AS disponivel,
	SUM(vlutilizado) AS utilizado,
	(SUM(vldisponivel) + SUM(vlutilizado)) AS total
FROM total
GROUP BY data_base
ORDER BY data_base ASC;'''
#print(sql)

limite = sqldf(sql)
limite = adiciona_data_base(limite)
limite['data_base'] = pd.to_datetime(limite['data_base'], format='%Y-%m-%d')
limite_atual = limite[limite['data_base'] == dt]['total'].sum()
limite['total'] = limite['total'].fillna(limite_atual)
limite['vl_esperado'] = None
limite['perc_esperado'] = None
limite['perc_utilizado'] = None
for i, r in limite.iterrows():
    mes = r['data_base'].month
    fracao_limite = round(r['total'] / 12 * mes)
    perc_fracao_limite = fracao_limite / r['total'] * limite_suplementacao_loa
    limite.at[i, 'vl_esperado'] = fracao_limite
    limite.at[i, 'perc_esperado'] = perc_fracao_limite
    limite.at[i, 'perc_utilizado'] = r['utilizado'] / r['total'] * limite_suplementacao_loa
#limite

In [ ]:
df = limite.copy()
fig, ax = plt.subplots(figsize=figsize)
fig.suptitle(nome, x=0, fontsize=16)
x = df['data_base'].dt.strftime('%b')
plt.plot(x, df['total'], color=cgrey, label='Autorizado', linestyle='--')
plt.plot(x, df['vl_esperado'], color=corange, label='Esperado')
plt.bar(x, df['disponivel'], color=cgreen, label='Disponível')
plt.bar(x, df['utilizado'], color=cblue, label='Utilizado')
plt.ylim(bottom=0, top=ax.get_ylim()[1] * 1.2)
ax.yaxis.set_major_formatter(int_formatter)
plt.xticks(x)
plt.legend(loc='upper right', ncol=3)
plt.show()


In [ ]:
lspec = {
    'data_base': 'Mês',
    'total': 'Autorizado',
    'utilizado': 'Utilizado',
    'disponivel': 'Disponível',
    'perc_esperado': '% esperado',
    'perc_utilizado': '% utilizado'
}
lfmt = {
    'data_base': date_to_month_name,
    'total': money_formatter,
    'utilizado': money_formatter,
    'disponivel': money_formatter,
    'perc_esperado': perc2_formatter,
    'perc_utilizado': perc2_formatter
}
df = df_table(limite.copy(), lspec, lfmt)
df.style.set_caption(f'{nome} - {dt.year}').hide(axis='index')